# Common Libraries

In [14]:
import gzip
import re
import xml.etree.ElementTree as ET
import pandas as pd

# Params

In [15]:
brodman_nifti_path = "/usr/local/afni/abin/Brodmann_pijn_afni.nii.gz"
save_xml_data_path = "/home/seojin/Seojin_commonTool/Module/Brain_ROI/metadata.xml"

is_save_xml = True
save_info_path = "/home/seojin/Seojin_commonTool/Module/Brain_ROI/Brodmann_pijn_afni_info.csv"

# Load data

In [16]:
with gzip.open(brodman_nifti_path, 'rt', errors='ignore') as f:
    content = f.read()

# Filtering meta data

In [17]:
metadata = re.findall(r'<AFNI_attributes.*?</AFNI_attributes>', content, re.DOTALL)[0]

if is_save_xml:
    with open(save_xml_data_path, 'w') as xml_file:
        xml_file.write(metadata)

# Read meta data

In [18]:
root = ET.fromstring(metadata)
value_label_dtable = None
for atr in root.findall(".//AFNI_atr"):
    if atr.get('atr_name') == "VALUE_LABEL_DTABLE":
        value_label_dtable = atr.text
        break

# Filtering roi info

In [20]:
labels = []
descriptions = []
for row in value_label_dtable.split("\n"):
    datas = row.split(" ")
    n_str = len(datas)

    if n_str > 1:
        label = datas[0].strip('"')
        description = datas[1].strip('"')
        
        if ("ctx" in description) or ("BA" in description):
            labels.append(label)
            descriptions.append(description)

In [21]:
empty_index = labels.index("")
descriptions[empty_index-1] = descriptions[empty_index - 1] + descriptions[empty_index]

In [18]:
del labels[empty_index]
del descriptions[empty_index]

# Save data

In [31]:
save_info = pd.DataFrame({ "label" : labels, "description" : descriptions })
save_info.to_csv(save_info_path, index=False)

In [33]:
save_info_path

'/home/seojin/Seojin_commonTool/Module/Brain_ROI/Brodmann_pijn_afni_info.csv'